In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Cargar los datos
data = pd.read_csv('cars.csv', sep=';')



In [5]:
# 1. LIMPIEZA DE DATOS

# Reemplazar valores "NO DATA" o similares con NaN
data.replace(["NO DATA", "N/A", "n/a", "?", "", " "], np.nan, inplace=True)

# Rellenar valores faltantes (media para numéricos, moda para categóricos)
for column in data.columns:
    if data[column].dtype == 'object':  # variables categóricas
        data[column].fillna(data[column].mode()[0], inplace=True)
    else:  # variables numéricas
        data[column].fillna(data[column].mean(), inplace=True)

# 1.2 Eliminación de outliers usando el rango intercuartil (IQR)
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Aplicamos el tratamiento de outliers solo a las columnas numéricas relevantes
numeric_cols = ['EDAD_COCHE', 'COSTE_VENTA', 'km_anno', 'Edad Cliente', 'Tiempo']
for col in numeric_cols:
    data = remove_outliers(data, col)

# 2. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
# Usamos LabelEncoder para convertir variables categóricas en formato numérico
categorical_cols = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Guardar el codificador para su uso futuro

# 3. NORMALIZACIÓN Y ESCALADO
# Escalado de las variables numéricas
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# Guardar el archivo ya procesado en un nuevo archivo CSV limpio
data.to_csv('cars_limpio.csv', index=False)

print("Archivo limpio guardado como 'ruta_a_tu_archivo_limpio.csv'")



Archivo limpio guardado como 'ruta_a_tu_archivo_limpio.csv'


In [6]:
#3. Desarrollo del Modelo de Propensión:
from sklearn.model_selection import train_test_split
data=pd.read_csv('cars_limpio.csv')
X = data.drop('Mas_1_coche', axis=1)  # Todas las variables menos el objetivo
y = data['Mas_1_coche']               # Variable objetivo

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
# Modelo de regresión logística
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score


logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_log = logistic_model.predict(X_test)
print(f"Accuracy (Regresión Logística): {accuracy_score(y_test, y_pred_log)}")
print(f"AUC-ROC: {roc_auc_score(y_test, logistic_model.predict_proba(X_test)[:, 1])}")
print(classification_report(y_test, y_pred_log))


ImportError: cannot import name 'accuracy_score' from 'sklearn.base' (c:\Users\Angel Martínez\anaconda3\lib\site-packages\sklearn\base.py)